# Trabajo Práctico 2: Pruebas de validez

Integrantes: 

Luis Paredes 104851

# Enunciado

El objetivo consiste en probar la validez de un programa que calcula $2^{8^{20}}$ módulo
p, utilizando diferentes estrategias y viendo el perfil de costos (tanto en términos de
longitud de la prueba como de tiempo de generación). 

1. Considerar 
    * $a_0 = 2$ 
    * $a_{n+1}=a_n^8$.
2. Considerar 
    * $a_0 = 2$ 
    * $a_{n+1} = a_n^2$
3. Considerar 
    * $a_0 = 2$ 
    * $a_{2n+1}=a_{2n}^2$ 
    * $a_{2n}=a_{2n-1}^4$.
4. Usando 2 columnas.

Tener en cuenta que, dado el grado de las restricciones, se requiere usar un factor de
expansión (blowup) por lo menos tan grande como el grado de las restricciones.

In [1]:
import channel, field, list_utils, merkle, polynomial

import time
from field import FieldElement # Cuerpo Finito
from polynomial import X # Polinomios
from polynomial import interpolate_poly # Interpolacion de Lagrange
from polynomial import prod # Producto de polinomios
from merkle import MerkleTree # Merkle Tree
from channel import Channel # Implements Fiat-Shamir Heuristic

from CP import CP_eval, get_CP # Returns a list of images from the CP
from FRI import FriCommit # Next domain for the FRI operator
from decommitment import decommit_fri # proof algorithm

Para todo el trabajo usaremos
* El primo $p = 3 \cdot 2^{30} + 1 = 3221225473$
* Espacio Finitamente generado $F _{3 \cdot 2^{30} + 1}= F_{3221225473}$

# Afirmacion a probar

Quiero probar que conozco el valor de $2^{8^{20}}$ módulo p

 * $a_0 = 2$ 
 * $a_{n+1}=a_n^8$.

# 1. Genero la traza de ejecucion

Generamos unos cuantos valores de la tabla para tener una idea de como se ve

| a_i         | Valor     | 
|--------------|-----------|
| $a_0$ | 2      | 2        |
| $a_1$ | $2^8$ |
| $a_2$      | $2^{64}$ | 
| $a_3$      | $2^{512}$ |

Observamos que $a_1 = 2^{8^1}$ , $a_2 = 2^{8^2}$, $a_3 = 2^{8^3}$ por ende reescribo la ecuacion como

$$a_{n} = a_o^{8^n} = 2^{8^n}, n>0$$

por ende la tabla se vera asi

| $a_i$         | Valor     | 
|--------------|-----------|
| $a_0$        | $2$         |
| $a_1$        | $2^{8^{1}}$ | 
| $a_2$        | $2^{8^{2}}$ |
| $a_3$        | $2^{8^{3}}$ |
| $...$        | $...$ |
|$a_{21}$      |  $2^{8^{20}}$  |

Genero la tabla de ejecucion para los 22 elementos que necesito

In [2]:
print("Generating the trace")

# Tabla de Ejecucion
a = [FieldElement(2)]

# a_{n} = 2^{8^n}
while len(a) < 22:
    a.append(a[0]**(8**(len(a))))

Generating the trace


In [3]:
a

[2,
 256,
 -1431655764,
 -601713034,
 284107013,
 -1284961466,
 948291965,
 -882704695,
 -1412552477,
 1610563585,
 1610563584,
 -1610563585,
 1610563584,
 -1610563585,
 1610563584,
 -1610563585,
 1610563584,
 -1610563585,
 1610563584,
 -1610563585,
 1610563584,
 -1610563585]

# 2. Busco un espacio finitamente generado   

Busco elemento generador $g$ tal que el subgrupo sea de orden n y genere el subgrupo 
$G = {g,g^1, g^2, ... , g^n}$, $G[i] = g^i$

Hint: When 𝑘 divides |𝔽×|, $𝑔^𝑘$ generates a group of size |𝔽×| / 𝑘, 
and the n-th power of some FieldElement 𝑥 can be computed by calling x ** n.

---

Queremos que el orden del subgrupo G sea de 32 porque es la potencia de 2 mas pequeña que excede a la traza.

|𝔽×| / 𝑘 = 32

Sabiendo que |𝔽×| = 3221225473

Tomando $k = 100663296$


Tal que 

|𝔽×| / 𝑘 $\approx 32$.

In [4]:
# Nuestro grupo es de orden 32

# Siendo n el orden del subgrupo
# n*k = |𝔽×| , busco k tal que |𝔽×| = 3221225473
n = 32
mod_F = FieldElement.k_modulus
k = mod_F // n

g = FieldElement.generator() ** (k)  # Busco g^k
G = [g ** i for i in range(n)] # Genero el subgrupo de orden 32

# 3. Busco un polinomio asociado a la traza y hago interpolacion extendiendo la traza

Ahora con el polinomio asociado, cada elemento de la traza lo veo como una evaluacion hecha del polinomio $f$ sobre
elementos $g$ (del espacio generador). Esto se llama Reed-Solomon error correction code.

Ahora la traza se ve asi

|$f(g^i)$| $a_i$         | Valor     | 
|----    |--------------|-----------|
|$f(g^0)$ | $a_0$        | $2$         |
|$f(g^1)$ | $a_1$        | $2^{8^{1}}$ | 
|$f(g^2)$ | $a_2$        | $2^{8^{2}}$ |
|$f(g^3)$ | $a_3$        | $2^{8^{3}}$ |
| $...$   | $...$        | $...$ |
|$f(g^{21})$ |$a_{21}$      |  $2^{8^{20}}$  |

Evaluamos en un dominio mucho mas grande que G. Tomamos un dominio H que es 8 veces mas grande

 n = 8*32

 |𝔽×| = 3221225473
 
 $k \approx 12582912$

In [5]:
print("We blow up the trace (Extend the trace) 8 times bigger")

n = 8*32
mod_F = FieldElement.k_modulus
k = mod_F // n

w = FieldElement.generator()

h = w ** (k)  # Este es mi generador g^k para el nuevo dominio
H = [h ** i for i in range(n)] # Genero el subgrupo de orden 8*32

eval_domain = [w * x for x in H]

We blow up the trace (Extend the trace) 8 times bigger


# 4. Busco un polinomio asociado y hago interpolacion

In [6]:
f = interpolate_poly(G[:len(a)], a)  ## Este polinomio pasa por todos los puntos que nos interes
f_eval = [f(d) for d in eval_domain] ## Busco la imagen en todos los elementos del dominio extendido

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 5816.24it/s]


# 5. Commitment 

Con la traza extendida, ahora creo las hojas del merkle tree a partir de estos 

In [7]:
f_merkle = MerkleTree(f_eval) 

# 6. Channels

Teoricamente el protocolo STARK requiere que dos personas (prover y verifier) interactuen entre si.

En la practica convertimos este protocolo interactivo a uno no interactivo haciendo uso de la heuristica de Fiat-Shamir.

In [8]:
channel = Channel()     # El channel nos dara datos que deberia darnos el verifier
channel.send(f_merkle.root)

El channel va a actuar como nuestro verificador de ahora en adelante. Nos va a proveer con cualquier dato requerido de parte del mismo.

# 7. Generamos CP y FRI layers

Convierto todas las restricciones originales a rational functions.

* $a_0 = 2$

    1) $f(x) = 2$, para $x=g^0$
    
    2) $g^0$ es raiz de $f(x)-2$
    
    3) $p_o(x) = \frac{f(x) - 2}{x-g^0}$

In [9]:
## a0 = 2
numer0 = f - 2
denom0 = X - 1
p0 = numer0 / denom0

* $a_{n+1}=a_n^8$

    1) $f(g\cdot x) = f(x)^8$
    
    2) $\{g^i \, , 0 \leq i \leq 20 \}$ es raiz de $f(g\cdot x)- f(x)^8$
    
    3) $p_1(x) = \frac{f(g\cdot x)-f(x)^8}{\prod_{i=0}^{20}(x-g^i)}$

In [10]:
# a[n+1] = (an)^8 

numer1 = f(g* X) - f(X)**8
 

lst = [(X - g**i) for i in range(21)]
denom1 = prod(lst)

p1 = numer1 / denom1

Genero el Composition Polynomial como combinacion lineal de estos polinomios.

Los coeficientes del mismo me los provee el channel ya que toma el roll de verificador

In [11]:
cp = get_CP(channel,[p0,p1])
cp_eval = CP_eval(cp,eval_domain)
cp_merkle = MerkleTree(cp_eval)
channel.send(cp_merkle.root)

In [12]:
cp

FRI Folding Operator

In [13]:
fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)

# 8. Genero queries y decommitments

In [14]:
len_query = len(eval_domain) - 1
decommit_fri(channel, fri_layers, fri_merkles, f_eval, f_merkle, len_query)

# 9. Vemos la prueba

In [15]:
print(channel.proof)
print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')

['send:014b16f1677633408f2b59e509cedf299158af8680caac7a2ed0bc34acf8356c', 'receive_random_field_element:467112066', 'receive_random_field_element:1990644572', 'send:546b947dc0fb7cf40e05c26ad8af6f08e317054d0b1a6b15e1dc1ca7344e07ad', 'receive_random_field_element:162423476', 'send:ef999a02386ca068d73fc4d11cf68e445998d90640d97e90a703712f34b6dc17', 'receive_random_field_element:908382570', 'send:6f8d01ceda157f7b594a8cf006976e1e9b15703a6404cd7edf2bcb95c1903432', 'receive_random_field_element:1080691931', 'send:1bd7c98b0efd801d62ed9c8ad4895be0f6fece3bab8e76840caca001ff254b33', 'receive_random_field_element:1077015045', 'send:1fdfb91e8f5f65f0007649cec1f5d844fab1d003610f4ac783f99d56ea2b4d60', 'receive_random_field_element:1091912111', 'send:ec24c61d5e805be7315add859ffc6ee728dfd0ad8c5e8c8903b451a5a471e258', 'send:-908778102', 'receive_random_int:236', 'send:-1184135388', "send:['5a9e9341a4b7658d1011569501f545fc0f2046050cc5d2ba881005d27914cf87', '83c8466c4d0a6e091fe7e79b08f2fec0645d81ddc260b563b

Uncompressed proof length in characters: 19178

# Tiempos

Vuelvo a correr toda la prueba e imprimo en pantalla el tiempo que toma correr cada parte

In [16]:
def modelo_uno():
    
    start = time.time()
    start_all = start
    print("Generating the trace")

    # Tabla de Ejecucion
    a = [FieldElement(2)]

    # a_{n} = 2^{8^n}
    while len(a) < 22:
        a.append(a[0]**(8**(len(a))))

    
    # Siendo n el orden del subgrupo
    # n*k = |𝔽×| , busco k tal que |𝔽×| = 3221225473
    n = 32
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    g = FieldElement.generator() ** (k)  # Busco g^k
    G = [g ** i for i in range(n)] # Genero el subgrupo de orden 32
    

    print("We blow up the trace (Extend the trace) 8 times bigger")
    
    n = 8*32
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    w = FieldElement.generator()

    h = w ** (k)  # Este es mi generador g^k para el nuevo dominio
    H = [h ** i for i in range(n)] # Genero el subgrupo de orden 8*32

    eval_domain = [w * x for x in H]

    f = interpolate_poly(G[:len(a)], a)  ## Este polinomio pasa por todos los puntos que nos interes
    f_eval = [f(d) for d in eval_domain] ## Busco la imagen en todos los elementos del dominio extendido


    f_merkle = MerkleTree(f_eval) 

    channel = Channel()     # El channel nos dara datos que deberia darnos el verifier
    channel.send(f_merkle.root)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating the composition polynomial and the FRI layers...")

    ## a0 = 2
    numer0 = f - 2
    denom0 = X - 1
    p0 = numer0 / denom0


    # a[n+1] = (an)^8 
    numer1 = f(g* X) - f(X)**8

    lst = [(X - g**i) for i in range(21)]
    denom1 = prod(lst)
    p1 = numer1 / denom1


    cp = get_CP(channel,[p0,p1])
    cp_eval = CP_eval(cp,eval_domain)
    cp_merkle = MerkleTree(cp_eval)
    channel.send(cp_merkle.root)

    # FRI Folding Operator
    fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating queries and decommitments...")

    len_query = len(eval_domain) - 1
    decommit_fri(channel, fri_layers, fri_merkles, f_eval, f_merkle, len_query)


    # Verificamos
    print(channel.proof)
    print(f'Overall time: {time.time() - start_all}s')
    print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')

    
if __name__ == "__main__":
    modelo_uno()

Generating the trace
We blow up the trace (Extend the trace) 8 times bigger


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 5998.87it/s]

0.01539468765258789s
Generating the composition polynomial and the FRI layers...
0.06260991096496582s
Generating queries and decommitments...


['send:014b16f1677633408f2b59e509cedf299158af8680caac7a2ed0bc34acf8356c', 'receive_random_field_element:467112066', 'receive_random_field_element:1990644572', 'send:546b947dc0fb7cf40e05c26ad8af6f08e317054d0b1a6b15e1dc1ca7344e07ad', 'receive_random_field_element:162423476', 'send:ef999a02386ca068d73fc4d11cf68e445998d90640d97e90a703712f34b6dc17', 'receive_random_field_element:908382570', 'send:6f8d01ceda157f7b594a8cf006976e1e9b15703a6404cd7edf2bcb95c1903432', 'receive_random_field_element:1080691931', 'send:1bd7c98b0efd801d62ed9c8ad4895be0f6fece3bab8e76840caca001ff254b33', 'receive_random_field_element:1077015045', 'send:1fdfb91e8f5f65f0007649cec1f5d844fab1d003610f4ac783f99d56ea2b4d60', 'receive_random_field_element:1091912111', 'send:ec24c61d5e805be7315add859ffc6ee728dfd0ad8c5e8c8903b451a5a471e258', 'send:-908778102', 'receive_random_int:236', 'send:-1184135388', "send:['5a9e9341a4b7658d1011569501f545fc0f2046050cc5d2ba881005d27914cf87', '83c8466c4d0a6e091fe7e79b08f2fec0645d81ddc260b563b

Vemos que tomamos
- 0.01599 segundos para crear la traza de ejecucion y extender el dominio 8 veces su tamaño
- 0.06235 segundos en generar el Composition Polynomial y hacer FRI layers
- 0.3503 segundos en total

Los tiempos suelen variar ligeramente entre compilaciones

# Modelo 2

Repetimos todo este analisis ahora con los siguientes modelos

2. Considerar 
    * $a_0 = 2$ 
    * $a_{n+1} = a_n^2$
    
Sabieno que $2^{8^{20}} = 2^{{(2^3)}^{20}} = 2^{2^{60}}$

|$f(g^i)$| $a_i$         | Valor     | 
|----    |--------------|-----------|
|$f(g^0)$ | $a_0$        | $2$         |
|$f(g^1)$ | $a_1$        | $2^{2^{1}}$ | 
|$f(g^2)$ | $a_2$        | $2^{2^{2}}$ |
|$f(g^3)$ | $a_3$        | $2^{2^{3}}$ |
| $...$   | $...$        | $...$ |
|$f(g^{60})$ |$a_{60}$   |  $2^{2^{60}}$|

La tabla de ejecucion tendra 61 elementos por lo que se tomara un subgrupo de orden 64.

Los polinomios asociados son

 * $a_0 = 2$
 
    1) $f(x) = 2$, para $x=g^0$
    
    2) $g^0$ es raiz de $f(x)-2$
    
    3) $p_0(x) = \frac{f(x) - 2}{x-g^0}$
 

* $a_{n+1} = a_n^2$

   1) $f(g\cdot x) = f(x)^2$
    
    2) $\{g^i \, , 0 \leq i \leq 59 \}$ es raiz de $f(g\cdot x)- f(x)^2$
    
    3) $p_1(x) = \frac{f(g\cdot x)-f(x)^2}{\prod_{i=0}^{59}(x-g^i)}$



In [17]:
def modelo_dos():
    
    start = time.time()
    start_all = start
    print("Generating the trace")

    # Tabla de Ejecucion
    a = [FieldElement(2)]

    # a_{n} = 2^{8^n}
    while len(a) < 60:
        a.append(a[0]**(2**(len(a))))

    
    # Siendo n el orden del subgrupo
    # n*k = |𝔽×| , busco k tal que |𝔽×| = 3221225473
    n = 64
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    g = FieldElement.generator() ** (k)  # Busco g^k
    G = [g ** i for i in range(n)] # Genero el subgrupo de orden 64
    

    print("We blow up the trace (Extend the trace) 8 times bigger")
    
    n = 8*64
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    w = FieldElement.generator()

    h = w ** (k)  # Este es mi generador g^k para el nuevo dominio
    H = [h ** i for i in range(n)] # Genero el subgrupo de orden 8*64

    eval_domain = [w * x for x in H]

    f = interpolate_poly(G[:len(a)], a)  ## Este polinomio pasa por todos los puntos que nos interes
    f_eval = [f(d) for d in eval_domain] ## Busco la imagen en todos los elementos del dominio extendido


    f_merkle = MerkleTree(f_eval) 

    channel = Channel()     # El channel nos dara datos que deberia darnos el verifier
    channel.send(f_merkle.root)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating the composition polynomial and the FRI layers...")

    ## a0 = 2
    numer0 = f - 2
    denom0 = X - 1
    p0 = numer0 / denom0


    # a[n+1] = (an)^2
    numer1 = f(g* X) - f(X)**2

    lst = [(X - g**i) for i in range(59)]
    denom1 = prod(lst)
    p1 = numer1 / denom1


    cp = get_CP(channel,[p0,p1])
    cp_eval = CP_eval(cp,eval_domain)
    cp_merkle = MerkleTree(cp_eval)
    channel.send(cp_merkle.root)
    
    print("El polinomio CP es")
    display(cp)
    
    # FRI Folding Operator
    fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating queries and decommitments...")

    len_query = len(eval_domain) - 1
    decommit_fri(channel, fri_layers, fri_merkles, f_eval, f_merkle, len_query)


    # Verificamos
    print(channel.proof)
    print(f'Overall time: {time.time() - start_all}s')
    print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')

    
if __name__ == "__main__":
    modelo_dos()

Generating the trace
We blow up the trace (Extend the trace) 8 times bigger


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 2313.80it/s]

0.07666134834289551s
Generating the composition polynomial and the FRI layers...
El polinomio CP es


0.08289265632629395s
Generating queries and decommitments...
['send:f34a7d52b1013bcb96d9ed083c1d247c53bc81cf2f1c1178118f2e2f32c2940f', 'receive_random_field_element:598333599', 'receive_random_field_element:129547673', 'send:1e247ab320415e2ded18932129325d01e387341f6dbffd039bb43dacb2e71d47', 'receive_random_field_element:2293590799', 'send:157a4707acb15cb84cdb35a2daa264da654bf00e0a3cd3e04d9938974440a9c2', 'receive_random_field_element:2168478102', 'send:4147203138cbfdef5c3e46392fd7f67c1ad90414e77024d301b715a2985cf149', 'receive_random_field_element:1728928168', 'send:fa12893bc11abdc5dfc773dd3f1cc90954be0914e857b4729ded1f47534ddc93', 'receive_random_field_element:1608258013', 'send:af9689b6d4060b04641591008553e08325f5ec387c578bf5662e61be1e6159d8', 'receive_random_field_element:1838309686', 'send:47ba4bbfbfb14f71550ce9fc35df4db6648b8888f38599a0a388c072d228d510', 'receive_random_field_element:29148827', 'send:4839936eab97c5bbe16f75cf3934e4777aad15d948f92cb146db6491abc0e4f6', 'send:15980913

In [18]:
cp

Obtenermos al final Uncompressed proof length in characters: 23748

Vemos que tomamos
- 0.075175 segundos para crear la traza de ejecucion y extender el dominio 8 veces su tamaño
- 0.077137 segundos en generar el Composition Polynomial y hacer FRI layers
- 0.442337 segundos en total

Observamos que tiene un desempeño peor que el modelo anterior. Esto se debe a que al ser la tabla de ejecucion mas larga entonces el problema es mas largo en general ya que el polinomio asociado va a ser de grado mayor.

# Modelo 3

2. Considerar 
    * $a_0 = 2$ 
    * $a_{2n+1}=a_{2n}^2$ 
    * $a_{2n}=a_{2n-1}^4$.
    
Sabieno que $2^{8^{20}}$

|$f(g^i)$| $a_i$         | Valor     
| :----:    | :--------------: |:------------
|$f(g^0)$ | $a_0$        | $2$ |
|$f(g^1)$ | $a_1$        | $2^{2}$ | 
|$f(g^2)$ | $a_2$        | $2^{8}$ |
|$f(g^3)$ | $a_3$        | $2^{8*2}$ |
|$f(g^4)$ | $a_4$        | $2^{8^2}$ |
|$f(g^5)$ | $a_5$        | $2^{8^2*2}$ |
|$f(g^6)$ | $a_6$        | $2^{8^{3}}$ |
|$f(g^7)$ | $a_7$        | $2^{8^{3}*2}$ |
|$f(g^8)$ | $a_8$        | $2^{8^{4}}$ |
| $...$   | $...$        | $...$ |
|$f(g^{40})$ |$a_{40}$   |  $2^{2^{8^{20}}}$|

Observo el patron 

* $a_{2n+1}=2^{8^n \cdot 2}$

* $a_{2n}=2^{8^n}$

De ahi es inmediato que $2^{8^n} = 2^{8^{20}} \rightarrow n=40$ Por ende $a_{40} = 2^{8^{20}}$


La tabla de ejecucion tendra 41 elementos por lo que se tomara un subgrupo de orden 64.

Los polinomios asociados son

 * $a_0 = 2$
 
    1) $f(x) = 2$, para $x=g^0$
    
    2) $g^0$ es raiz de $f(x)-2$
    
    3) $p_0(x) = \frac{f(x) - 2}{x-g^0}$
 

* $a_{2n+1}=a_{2n}^2$ 

* $a_{2n}=a_{2n-1}^4$.

Nos encontramos con que la restriccion 2) se aplica para los n par y la restriccion 3) solo para n impar. 

Tenemos que usar [trucos matematicos](https://blog.lambdaclass.com/periodic-constraints-and-recursion-in-zk-starks/) para llegar una expresion que nos sirva

### Terminar este punto

In [20]:
def modelo_tres():
    
    start = time.time()
    start_all = start
    print("Generating the trace")

   # Tabla de Ejecucion
    a = [FieldElement(2)] # a0 = 2

    # a_{n} = 2^{8^n}
    while len(a) < 41:
        a.append(a[-1]**2) # 𝑎_{2𝑛+1}=𝑎_{2𝑛}^2
        a.append(a[-1]**4) # 𝑎_{2𝑛}=𝑎_{2𝑛−1}^4
    
    # Siendo n el orden del subgrupo
    # n*k = |𝔽×| , busco k tal que |𝔽×| = 3221225473
    n = 64
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    g = FieldElement.generator() ** (k)  # Busco g^k
    G = [g ** i for i in range(n)] # Genero el subgrupo de orden 64
    

    print("We blow up the trace (Extend the trace) 8 times bigger")
    
    n = 8*64
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    w = FieldElement.generator()

    h = w ** (k)  # Este es mi generador g^k para el nuevo dominio
    H = [h ** i for i in range(n)] # Genero el subgrupo de orden 8*64

    eval_domain = [w * x for x in H]

    f = interpolate_poly(G[:len(a)], a)  ## Este polinomio pasa por todos los puntos que nos interes
    f_eval = [f(d) for d in eval_domain] ## Busco la imagen en todos los elementos del dominio extendido


    f_merkle = MerkleTree(f_eval) 

    channel = Channel()     # El channel nos dara datos que deberia darnos el verifier
    channel.send(f_merkle.root)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating the composition polynomial and the FRI layers...")

    ## a0 = 2
    numer0 = f - 2
    denom0 = X - 1
    p0 = numer0 / denom0

    
    ##### TODO

    # 𝑎[2𝑛+1]=𝑎[2𝑛]^2
    numer1 = f(g* X) - f(X)**2

    lst = [(X - g**i) for i in range(41)]
    denom1 = prod(lst)
    p1 = numer1 / denom1
        
    ##### TODO
    
    # 𝑎[2𝑛]=𝑎[2𝑛−1]^4
    numer2 = f(g* X) - f(X)**2  # TODO

    lst = [(X - g**i) for i in range(41)]
    denom2 = prod(lst)
    
    p2 = numer2 / denom2

    cp = get_CP(channel,[p0,p1,p2])
    cp_eval = CP_eval(cp,eval_domain)
    cp_merkle = MerkleTree(cp_eval)
    channel.send(cp_merkle.root)
    
    
    print("El polinomio CP es")
    display(cp)

    # FRI Folding Operator
    fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating queries and decommitments...")

    len_query = len(eval_domain) - 1
    decommit_fri(channel, fri_layers, fri_merkles, f_eval, f_merkle, len_query)


    # Verificamos
    print(channel.proof)
    print(f'Overall time: {time.time() - start_all}s')
    print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')

    
if __name__ == "__main__":
    modelo_tres()

Generating the trace
We blow up the trace (Extend the trace) 8 times bigger


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 3134.30it/s]

0.03607034683227539s
Generating the composition polynomial and the FRI layers...


AssertionError: Polynomials are not divisible.

# Modelo 4 (To-Do)

Usando 2 columnas.

En lugar de tener todos los valores en una lista, tenerla en dos. Podés tener dos columnas a y b, donde b(x)=a(x)^2 y a(gx)=b(x)^4

In [ ]:
def modelo_cuatro():
    
    start = time.time()
    start_all = start
    print("Generating the trace")

   # Tabla de Ejecucion
    a = [FieldElement(2)] # a0 = 2

    # a_{n} = 2^{8^n}
    while len(a) < 41:
        a.append(a[-1]**2) # 𝑎_{2𝑛+1}=𝑎_{2𝑛}^2
        a.append(a[-1]**4) # 𝑎_{2𝑛}=𝑎_{2𝑛−1}^4
    
    # Siendo n el orden del subgrupo
    # n*k = |𝔽×| , busco k tal que |𝔽×| = 3221225473
    n = 64
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    g = FieldElement.generator() ** (k)  # Busco g^k
    G = [g ** i for i in range(n)] # Genero el subgrupo de orden 64
    

    print("We blow up the trace (Extend the trace) 8 times bigger")
    
    n = 8*64
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    w = FieldElement.generator()

    h = w ** (k)  # Este es mi generador g^k para el nuevo dominio
    H = [h ** i for i in range(n)] # Genero el subgrupo de orden 8*64

    eval_domain = [w * x for x in H]

    f = interpolate_poly(G[:len(a)], a)  ## Este polinomio pasa por todos los puntos que nos interes
    f_eval = [f(d) for d in eval_domain] ## Busco la imagen en todos los elementos del dominio extendido


    f_merkle = MerkleTree(f_eval) 

    channel = Channel()     # El channel nos dara datos que deberia darnos el verifier
    channel.send(f_merkle.root)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating the composition polynomial and the FRI layers...")

    ## a0 = 2
    numer0 = f - 2
    denom0 = X - 1
    p0 = numer0 / denom0


    # 𝑎[2𝑛+1]=𝑎[2𝑛]^2
    numer1 = f(g* X) - f(X)**2  # TODO

    lst = [(X - g**i) for i in range(41)]
    denom1 = prod(lst)
    p1 = numer1 / denom1
    
    
    # 𝑎[2𝑛]=𝑎[2𝑛−1]^4
    numer2 = f(g* X) - f(X)**2  # TODO

    lst = [(X - g**i) for i in range(41)]
    denom2 = prod(lst)
    
    p2 = numer2 / denom2

    cp = get_CP(channel,[p0,p1,p2])
    cp_eval = CP_eval(cp,eval_domain)
    cp_merkle = MerkleTree(cp_eval)
    channel.send(cp_merkle.root)
    
    
    print("El polinomio CP es")
    display(cp)

    # FRI Folding Operator
    fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating queries and decommitments...")

    len_query = len(eval_domain) - 1
    decommit_fri(channel, fri_layers, fri_merkles, f_eval, f_merkle, len_query)


    # Verificamos
    print(channel.proof)
    print(f'Overall time: {time.time() - start_all}s')
    print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')

    
if __name__ == "__main__":
    modelo_cuatro()

# Conclusion

El protocolo STARK es una protocolo que nos permite demostrar o validar el trabajo hecho sin tener que hacerlo de vuelta o entregar datos personales que demuestren lo que afirmamos.

Con las pruebas de validez podemos probar que un cómputo se ha realizado correctamente en un tiempo bastante menor que la verificación trivial, es decir, que el cómputo sea corrido de manera independiente por el verificador. 

Ignorando un momento toda la matematica de trasfondo, el protocolo actua casi como magia y el hecho de que no requiera de interaccion externa le da una seguridad enorme. 

Sin embargo, con estas soluciones tambien creamos nuevos problemas. Como dependemos fuertemente del channel es critico que su aleatoridad sea absoluta y en el mundo real esto es algo que no podemos afirmar a ciegas. Si algun dia se descubre una manera de predecir los valores que nos da entonces todo el protocolo se romperia. 

Pero como todo en criptografia, el protocolo funciona bien hasta que inevitablemente alguien lo rompa. Por el momento, STARK parece ser muy prometedor para el futuro de la criptografia y ya hay planes para incorporarlo como una mejora de su protocolo hermano SNARK.